# Récupération des données de catastrophes naturelles avec Georisques


Lien de l'API utilisée :
https://georisques.gouv.fr/api/v1/gaspar/catnat



In [ ]:
import requests
import pandas as pd
import time
import gdown
import os

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Définition du chemin d'accès au dossier
path_cat_nat = 'drive/My Drive/PIP 2025 /Catastrophes naturelles/Catastrophes Géorisques/'
path_clean = 'drive/My Drive/PIP 2025 /Catastrophes naturelles/Données clean/'

In [ ]:
# URL de base de l'API Géorisques pour les catastrophes naturelles (CatNat)
API_URL = "https://www.georisques.gouv.fr/api/v1/gaspar/catnat"


# Test avec une seule commune

On commence par tester l'API avec une seule commune et on exporte le résultat en csv.

In [ ]:
# Paramètres de la requête (exemple : Toulouse, code INSEE = 31555)
params = {
    "code_insee": "31555",  # Code INSEE de la commune
    "categorie": "risques_naturels",
    "page": 1  # On commence à la première page
}

In [ ]:
# Liste pour stocker les données de toutes les pages
all_results = []

while True:
    # Requête GET à l'API
    response = requests.get(API_URL, params=params)

    # Vérifier si la requête est réussie
    if response.status_code == 200:
        data = response.json()

        # Ajouter les résultats de la page courante à la liste
        all_results.extend(data.get("data", []))

        # Vérifier s'il y a une page suivante
        next_page_url = data.get("next")
        if next_page_url:
            params["page"] += 1  # Passer à la page suivante
        else:
            break  # Sortie de la boucle si pas de page suivante
    else:
        print(f"Erreur {response.status_code}: {response.text}")
        break

# Convertir les résultats en DataFrame pandas
df = pd.DataFrame(all_results)

# Afficher les 5 premières lignes
print(df.head())

# Sauvegarder en CSV
df.to_csv("catastrophes_naturelles_toulouse.csv", index=False, encoding="utf-8")

print("Données enregistrées dans 'catastrophes_naturelles.csv'")


  code_national_catnat date_debut_evt date_fin_evt date_publication_arrete  \
0         INTE0000364A     10/06/2000   10/06/2000              21/07/2000   
1         INTE0000771A     01/01/1998   31/12/1998              27/12/2000   
2         INTE0000771A     01/01/1998   31/12/1999              27/12/2000   
3         INTE0000771A     01/03/1999   31/12/1999              27/12/2000   
4         INTE0000771A     01/04/1999   31/12/1999              27/12/2000   

  date_publication_jo                  libelle_risque_jo code_insee  \
0          01/08/2000  Inondations et/ou Coulées de Boue      31555   
1          29/12/2000                         Sécheresse      31555   
2          29/12/2000                         Sécheresse      31555   
3          29/12/2000                         Sécheresse      31555   
4          29/12/2000                         Sécheresse      31555   

  libelle_commune  
0        TOULOUSE  
1        TOULOUSE  
2        TOULOUSE  
3        TOULOUSE  
4   

# Catastrophes naturelles pour les départements 01-09

Il est nécessaire d'ajouter un '0' devant les codes INSEE de ces département (car ils contiennent seulement 4 chiffres en récupérant le CSV) pour envoyer les requêtes à l'API avec le code correct. Cela permet de faire un premier test avec un nombre important de lignes à ramener également.

In [ ]:
# Charger le fichier CSV
df_code_insee_01_09 = pd.read_csv(path_cat_nat+'code_insee_4ch.csv', dtype={"code_insee": str})

In [ ]:
all_results = []

In [ ]:
df_code_insee_01_09

,code_insee
0,1001
1,1002
2,1004
3,1005
4,1006
...,...
3134,9338
3135,9339
3136,9340
3137,9341


In [ ]:
df_code_insee_01_09['code_insee'] = '0' + df_code_insee_01_09['code_insee']
df_code_insee_01_09

,code_insee
0,01001
1,01002
2,01004
3,01005
4,01006
...,...
3134,09338
3135,09339
3136,09340
3137,09341


In [ ]:
# Boucle sur chaque code INSEE du fichier CSV
# 33min d'exécution

for code_insee in df_code_insee_01_09["code_insee"]:
    #print(f"Récupération des données pour la commune {code_insee}...")

    params = {
        "code_insee": code_insee,  # Code INSEE de la commune
        "categorie": "risques_naturels",
        "page": 1  # On commence à la première page
    }

    while True:
        # Requête GET à l'API
        response = requests.get(API_URL, params=params)

        # Vérifier si la requête est réussie
        if response.status_code == 200:
            data = response.json()

            # Ajouter les résultats de la page courante à la liste
            all_results.extend(data.get("data", []))

            # Vérifier s'il y a une page suivante
            next_page_url = data.get("next")
            if next_page_url:
                params["page"] += 1  # Passer à la page suivante
            else:
                break  # Sortie de la boucle si pas de page suivante
        else:
            print(f"Erreur {response.status_code} pour la commune {code_insee}: {response.text}")
            break

    print(f'\rLe code INSEE suivant a été traité : {params["code_insee"]} Lignes traitées : {len(all_results)}',end='')
    #print(f'Données : {all_results}')


Le code INSEE suivant a été traité : 09342 Lignes traitées : 18453

In [ ]:
 # Convertir les résultats en DataFrame pandas
df_results_01_09 = pd.DataFrame(all_results)
df_results_01_09

,code_national_catnat,date_debut_evt,date_fin_evt,date_publication_arrete,date_publication_jo,libelle_risque_jo,code_insee,libelle_commune
0,IOCE0911363A,06/02/2009,07/02/2009,18/05/2009,21/05/2009,Inondations et/ou Coulées de Boue,01001,L'ABERGEMENT-CLEMENCIAT
1,NOR19840921,21/06/1984,21/06/1984,21/09/1984,18/10/1984,Inondations et/ou Coulées de Boue,01001,L'ABERGEMENT-CLEMENCIAT
2,INTE0000591A,14/06/2000,14/06/2000,25/10/2000,15/11/2000,Inondations et/ou Coulées de Boue,01002,L'ABERGEMENT-DE-VAREY
3,INTE9000113A,13/02/1990,18/02/1990,16/03/1990,23/03/1990,Inondations et/ou Coulées de Boue,01002,L'ABERGEMENT-DE-VAREY
4,INTE0400656A,01/07/2003,30/09/2003,25/08/2004,26/08/2004,Sécheresse,01004,AMBERIEU-EN-BUGEY
...,...,...,...,...,...,...,...,...
18448,INTE9300655A,21/09/1993,25/09/1993,29/11/1993,15/12/1993,Inondations et/ou Coulées de Boue,09342,SAINTE-SUZANNE
18449,IOCE0759144A,25/05/2007,26/05/2007,03/07/2007,10/07/2007,Inondations et/ou Coulées de Boue,09342,SAINTE-SUZANNE
18450,IOME2313528A,30/06/2022,29/09/2022,20/07/2023,07/09/2023,Sécheresse,09342,SAINTE-SUZANNE
18451,NOR19821118,06/11/1982,10/11/1982,18/11/1982,19/11/1982,Inondations et/ou Coulées de Boue,09342,SAINTE-SUZANNE


# Catastrophes naturelles pour les départements 10-95 (et Corse)

In [ ]:
# Charger le fichier CSV
df_code_insee_10_95 = pd.read_csv(path_cat_nat+'code_insee_5ch.csv', sep=';',dtype={"code_insee": str})

In [ ]:
df_code_insee_10_95

,code_insee
0,10002
1,10003
2,10004
3,10005
4,10006
...,...
31707,95676
31708,95678
31709,95680
31710,95682


In [ ]:
# Découper le DataFrame en sous-groupes selon le premier chiffre du code INSEE
df_grouped = {}

for i in range(1, 10):  # 1, 2, 3, ..., 9 (correspond à 10-19, 20-29, etc.)
    df_filtered = df_code_insee_10_95[df_code_insee_10_95["code_insee"].astype(str).str[0] == str(i)]

    if not df_filtered.empty:
        df_grouped[f"df_code_insee_{i}0_{i}9"] = df_filtered

# Vérification des groupes créés
for key, df in df_grouped.items():
    print(f"{key}: {len(df)} lignes")

df_code_insee_10_19: 3448 lignes
df_code_insee_20_29: 4324 lignes
df_code_insee_30_39: 4126 lignes
df_code_insee_40_49: 2666 lignes
df_code_insee_50_59: 4743 lignes
df_code_insee_60_69: 4814 lignes
df_code_insee_70_79: 3760 lignes
df_code_insee_80_89: 3230 lignes
df_code_insee_90_99: 601 lignes


In [ ]:
# 🟢 Sélectionne un groupe spécifique à traiter
selected_group = "df_code_insee_60_69"  # Change ce nom selon le DataFrame à traiter

if selected_group in df_grouped:
    df_subset = df_grouped[selected_group]  # Récupère le DataFrame choisi

    print(f"\n Traitement du groupe : {selected_group}")

    all_results = []

    for code_insee in df_subset["code_insee"]:
        params = {
            "code_insee": code_insee,
            "categorie": "risques_naturels",
            "page": 1
        }

        while True:
            # Requête GET à l'API
            response = requests.get(API_URL, params=params)

            # Vérifier si la requête est réussie
            if response.status_code == 200:
                data = response.json()

                # Ajouter les résultats de la page courante à la liste
                all_results.extend(data.get("data", []))

                # Vérifier s'il y a une page suivante
                next_page_url = data.get("next")
                if next_page_url:
                    params["page"] += 1  # Passer à la page suivante
                else:
                    break  # Sortie de la boucle si pas de page suivante
            else:
                print(f"❌ Erreur {response.status_code} pour la commune {code_insee}: {response.text}")
                break

        print(f'\rLe code INSEE suivant a été traité : {params["code_insee"]} Lignes traitées : {len(all_results)}',end='')




 Traitement du groupe : df_code_insee_60_69
Le code INSEE suivant a été traité : 69389 Lignes traitées : 32680

In [ ]:
# Convertir en DataFrame
df_results_60_69 = pd.DataFrame(all_results)

# Extraction du fichier

In [ ]:
df_results = df_results_60_69 #+ df_results....

In [ ]:
# Exporter le DataFrame en CSV
df_results.to_csv(path_clean+"catastrophes_naturelles_60_69.csv", index=False, encoding='utf-8-sig')

print("Extraction terminée ! Données enregistrées dans 'catastrophes_naturelles.csv'")

Extraction terminée ! Données enregistrées dans 'catastrophes_naturelles.csv'
